In [434]:
import pandas as pd
DAYS = 6
LESSONS_PER_DAY = 7
day_of_week_prefixes = [
    "mon",
    "tue",
    "wed",
    "thu",
    "fri",
    "sat"
]
cabinets_df = pd.read_csv("./data/cabinets.csv")
groups_df = pd.read_csv("./data/groups.csv")
professors_df = pd.read_csv("./data/professors.csv")
subjects_df = pd.read_csv("./data/subjects.csv")

groups_df.rename(columns={
    "nr_persoane": "people_count"
}, inplace=True)

groups_df["subject_ids"] = groups_df["subject_ids"].str.split(',')
groups_df = groups_df.explode("subject_ids")
groups_df['subject_ids'] = pd.to_numeric(groups_df['subject_ids'], errors='coerce')
groups_df = groups_df.dropna(subset=['subject_ids'])
print(groups_df.head())

subjects_df.rename(columns={
    "unitate_curs": "course",
    "teorie": "theory",
    "practica": "practice",
    "lab": "labs",
    "TOTAL": "projects",
    "anul": "year",
    "semestru": "semester"
}, inplace= True)

# NaN is replaced too
subjects_df.drop(columns=["projects"], inplace=True)
subjects_df['semester'] = subjects_df['semester'].str.split(',')
subjects_df = subjects_df.explode('semester')
subjects_df = subjects_df.astype({"id": int, "year": int, "semester": int, "theory": int, "practice": int, "labs": int})

cabinets_df.rename(columns={
    "nr_persons": "cabinet_capacity",
    "id ": "id"
}, inplace=True)
cabinets_df["cabinet_capacity"].fillna(30, inplace=True)

one_hot_df = pd.DataFrame(professors_df["type"])

# Creating dummy variables using get_dummies
dummies = one_hot_df['type'].str.get_dummies(sep=',')

# Concatenating the dummy variables with the original DataFrame
one_hot_df = pd.concat([one_hot_df, dummies], axis=1)
professors_df.drop("type", axis=1, inplace=True)
professors_df = pd.concat([professors_df, one_hot_df], axis=1)
professors_df.rename(columns={
    "TEOR": "theory",
    "PRACT": "practice",
    "LAB": "labs",
}, inplace=True)

   id speciality language  people_count  subject_ids
0   1     AI-231       ro            31          1.0
0   1     AI-231       ro            31          3.0
0   1     AI-231       ro            31         10.0
0   1     AI-231       ro            31         11.0
0   1     AI-231       ro            31          4.0


Create the currently existent timetable into a matrix for each academic group.

TI-224 -> [

    // Monday

    [

        // 8:00 - 9:30

        [None, {}, {}, {}]

        // 9:45 - 11:15

        [None, {}, {}, {}]

        // 11:30 - 13:00

        [None, {}, {}, {}]

    ]
    
]

In [435]:
# get first year first semester only
subjects_df = subjects_df[(subjects_df["year"] == 1) & (subjects_df["semester"] == 1)]
print(subjects_df.head())

from dataclasses import dataclass

@dataclass
class Lesson:
    subject: str
    type: str
    groups: list
    professor: str
    cabinet: str

   id                                             course  theory  practice  \
0   1                                                AM1      30        30   
2   3                                               ALGA      30        15   
3   4                                                 PC      30        30   
4   5                                                PSA      30        15   
5   6  Etica și integritatea academică/ Psihologie in...      30         0   

   labs  year  semester  
0     0     1         1  
2     0     1         1  
3    30     1         1  
4    15     1         1  
5     0     1         1  


In [436]:
cabinet_schedule_df = pd.DataFrame.copy(cabinets_df)
cabinet_schedule_df["count"] = 0
# add cross product for days and each lesson
new_keys = {
    "day": list(list(range(0, DAYS)) for _ in range(0, cabinet_schedule_df.shape[0])),
    "lesson_number": list(list(range(0, LESSONS_PER_DAY)) for _ in range(0, cabinet_schedule_df.shape[0]))
}
cabinet_schedule_df = cabinet_schedule_df.assign(**new_keys)
cabinet_schedule_df = cabinet_schedule_df.explode("day", ignore_index=True)
cabinet_schedule_df = cabinet_schedule_df.explode("lesson_number", ignore_index=True)
availability_fields = [
    day_of_week_prefixes[row["day"]] + "_per_" + str(row["lesson_number"] + 1) for _, row in cabinet_schedule_df.iterrows()
]
cabinet_schedule_df["availability"] = availability_fields

def get_cabinets_to_professors_df():
    cabinets_to_professors_df = pd.DataFrame(columns=["cabinet_id", "professor_id"])
    for _, row in cabinet_schedule_df.iterrows():
        is_lab_cab = row["is_lab_cab"] == 1
        availability_field = row["availability"]
        available_professor_ids = professors_df[((professors_df["labs"] != 0) == is_lab_cab) & (professors_df[availability_field] == 1)]["id"]

        df = pd.DataFrame(columns=["cabinet_id", "professor_id"])
        df["professor_id"] = available_professor_ids
        df["cabinet_id"] = [row.iloc[0] for _ in range(0, available_professor_ids.shape[0])]
        cabinets_to_professors_df = pd.concat([cabinets_to_professors_df, df], ignore_index=True)
    return cabinets_to_professors_df
    
cabinets_to_professors_df = get_cabinets_to_professors_df()
print(cabinets_to_professors_df.head())

  cabinet_id professor_id
0        3-3            1
1        3-3            2
2        3-3            3
3        3-3            7
4        3-3            8


In [437]:
solution_df = pd.DataFrame(columns=["day", "lesson_number", "cabinet", "subject", "type", "groups", "professor"])

In [445]:
from typing import Dict, List

def step_wave_function_collapse(subjects_df: pd.DataFrame, groups_df: pd.DataFrame, professors_df: pd.DataFrame, cabinets_df: pd.DataFrame, cabinet_schedule_df: pd.DataFrame):
    superpositions = pd.DataFrame(columns=["day", "lesson_number", "cabinet", "subject", "type", "groups", "professor"])
    available_cabinets = cabinet_schedule_df[cabinet_schedule_df["count"] == 0] # should be more complex logic later
    available_cabinets = available_cabinets.merge(cabinets_to_professors_df, left_on="id", right_on="cabinet_id")
    available_cabinets_with_professors = available_cabinets.merge(professors_df, left_on=["professor_id"], right_on=["id"], suffixes=("_cabinet", "_professor"))

    # join subjects with professors that teach the subject and groups that learn the subject
    cabinets_with_professors_with_subjects = pd.merge(subjects_df, available_cabinets_with_professors, left_on="id", right_on="subject", suffixes=("_subject", "_professor"))
    cabinets_with_professors_with_subjects_with_groups = pd.merge(cabinets_with_professors_with_subjects, groups_df, left_on="id", right_on="subject_ids", suffixes=("_subject", "_group"))
    
    cabinet_combinations_lab = cabinets_with_professors_with_subjects_with_groups[
        (cabinets_with_professors_with_subjects_with_groups["labs_professor"] > 0) &
        (cabinets_with_professors_with_subjects_with_groups["labs_subject"] > 0)
    ]
    cabinet_combinations_lab["lesson_type"] = "lab"
    
    cabinet_combinations_theory = cabinets_with_professors_with_subjects_with_groups[
        (cabinets_with_professors_with_subjects_with_groups["theory_professor"] > 0) &
        (cabinets_with_professors_with_subjects_with_groups["theory_subject"] > 0)
    ]
    cabinet_combinations_theory["lesson_type"] = "theory"
    
    cabinet_combinations_practice = cabinets_with_professors_with_subjects_with_groups[
        (cabinets_with_professors_with_subjects_with_groups["practice_professor"] > 0) &
        (cabinets_with_professors_with_subjects_with_groups["practice_subject"] > 0)
    ]
    cabinet_combinations_practice["lesson_type"] = "practice"
    
    cabinet_combinations = pd.concat([cabinet_combinations_lab, cabinet_combinations_theory, cabinet_combinations_practice])
    print(cabinet_combinations.shape)
    print(cabinet_combinations.columns)
    # cabinet_combinations = cabinet_combinations[["day", "lesson_number", "cabinet", "id_subject", "lesson_type", "groups_group", "professor"]]


In [446]:
step_wave_function_collapse(subjects_df, groups_df, professors_df, cabinets_df, cabinet_schedule_df)

C:\Users\DVR\AppData\Local\Temp\ipykernel_29372\2454797883.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabinet_combinations_lab["lesson_type"] = "lab"
